In [1]:
import pandas as pd
from google.cloud import bigquery

df = pd.read_csv('ordenamiento.csv').sort_values('score',ascending=False).reset_index(drop=True)

df_trunc=df[(df['item_id']!=641416750) & (df['item_id']!=22351223)]

df_ord=pd.DataFrame(columns=list(df_trunc.columns))
df_ord=pd.concat([df_ord,df_trunc[0:1]],ignore_index=True)

In [2]:
%%time
def f(n):
    minimo=n
    for i in range(n,len(df)):
        if df.vertical[i]==df_ord.vertical[n-1] or len(set(df_ord.domain[n-3:n]).union(set(df.domain[i:i+1])))==1:
            minimo=i+1
        else:
            return minimo
        
for i in range(len(df))[1:]:
    if i==f(i):
        df_ord=pd.concat([df_ord,df[i:i+1]]).reset_index(drop=True)
    else:
        df_ord=pd.concat([df_ord,df[f(i):f(i)+1]]).reset_index(drop=True)
        df_ord[i+1:]=pd.concat([df_ord[i:f(i)],df_ord[f(i)+1:]]).reset_index(drop=True)

df_final=df_ord[:2].append(df.loc[df['item_id']==641416750]).append(df_ord[2:4]).append(df.loc[df['item_id']==22351223]).append(df_ord[4:])
df_final.reset_index(drop=True,inplace=True)

def h(n):
    minimo=n
    for i in range(n,len(df_final)):
        if df_final.loc[i,'category']!='HOME&DECO':
            minimo=i+1
        else:
            return minimo
    return n
        
for i in range(8,11):
    if h(i)!=i:
        df_final[i:]=df_final[h(i):h(i)+1].append(df_final[i:].drop([h(i)]))

df_final

CPU times: user 12 s, sys: 96.9 ms, total: 12.1 s
Wall time: 12.2 s


,item_id,vertical,category,domain,score
0,590602034,CE,ELECTRONICS,MLC-GAME_CONSOLES,0.9998
1,523534468,BEAUTY & HEALTH,PHARMACEUTICS,MLC-SURGICAL_AND_INDUSTRIAL_MASKS,0.9976
2,641416750,CE,MOBILE,MLC-CELLPHONES,0.9986
3,634352041,CE,MOBILE,MLC-CELLPHONES,0.9994
4,523534468,BEAUTY & HEALTH,PHARMACEUTICS,MLC-SURGICAL_AND_INDUSTRIAL_MASKS,0.9976
...,...,...,...,...,...
4996,911699995,HOME & INDUSTRY,TOOLS AND CONSTRUCTION,MLC-ELECTRIC_PRESSURE_WASHERS,0.0008
4997,539428631,CPG,FOODS,MLC-EDIBLE_PAPERS,0.0006
4998,489295848,APP & SPORTS,APPAREL,MLC-PANTS,0.0004
4999,489817797,CPG,PERSONAL HYGIENE,MLC-WET_BABY_WIPES,0.0002
